In [ ]:
import uuid
import json
import datetime 
import random
import time
import datetime
time_fmt="%Y-%m-%d %H:%M:%S"
hours = 24 * 30
date = datetime.datetime(2017, 1, 1, 8, 0)
data_file = "deepar_training.json"
fh = open(data_file, "w")
data = []
bp_sistolic = []
bp_diastolic = []
bp_time = date
for m in range(0,hours):
    noise  = round(40 * random.random())
    bp_sistolic.append(110+noise)
    bp_diastolic.append(70+noise)
sistolic = {"start": bp_time.strftime(time_fmt), "target": bp_sistolic, "cat": 0}
sistolic = json.dumps(sistolic) 
diastolic = {"start": bp_time.strftime(time_fmt), "target": bp_diastolic, "cat": 1}
diastolic = json.dumps(diastolic) 
print(sistolic,file=fh)
print(diastolic,file=fh)
fh.close()

In [ ]:
!wc -l "{data_file}"


In [104]:
!cp "{data_file}" "{test_file}"
!open "{data_file}"

In [105]:
import uuid
uuid = str(uuid.uuid4())

In [106]:
bucket         = 'bpdata_'+uuid
prefix         = 'sagemaker'
region = "us-east-1" 
train_key      = 'deepar_training.json'
test_key       = 'deepar_test.json'
train_prefix   = '{}/{}'.format(prefix, 'train')
test_prefix    = '{}/{}'.format(prefix, 'test')
output_prefix  = '{}/{}'.format(prefix, 'output')
output_path = 's3://{}/{}'.format(bucket, output_prefix)


In [107]:
!aws s3 mb s3://{bucket}

make_bucket: bpdata_d4d38b6c-f257-4d83-9881-fe88b8f98a7c


In [108]:
import sagemaker
sagemaker_session = sagemaker.Session()

In [109]:
train_path  = sagemaker_session.upload_data(train_key, bucket=bucket, key_prefix=train_prefix)
test_path   = sagemaker_session.upload_data(test_key,  bucket=bucket, key_prefix=test_prefix)

In [110]:
!aws s3 ls  s3://{bucket}/sagemaker/train/

2018-02-04 01:14:16      13458 deepar_training.json


In [111]:
containers = {
    'us-east-1': '522234722520.dkr.ecr.us-east-1.amazonaws.com/forecasting-deepar:latest',
    'us-east-2': '566113047672.dkr.ecr.us-east-2.amazonaws.com/forecasting-deepar:latest',
    'us-west-2': '156387875391.dkr.ecr.us-west-2.amazonaws.com/forecasting-deepar:latest',
    'eu-west-1': '224300973850.dkr.ecr.eu-west-1.amazonaws.com/forecasting-deepar:latest'
}

image_name = containers[region]
image_name

'522234722520.dkr.ecr.us-east-1.amazonaws.com/forecasting-deepar:latest'

In [112]:
role = "arn:aws:iam::030555009967:role/service-role/AmazonSageMaker-ExecutionRole-20180107T101850"

In [113]:
estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sagemaker_session,
    image_name=image_name,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.c4.8xlarge',
    base_job_name='bp-job',
    output_path=output_path
)

In [114]:
estimator

In [115]:
prediction_length = 24
hyperparameters = {
    "time_freq": 'H',
    "context_length": prediction_length,
    "prediction_length": prediction_length, # number of data points to predict
    "num_cells": "40",
    "num_layers": "2",
    "likelihood": "gaussian",
    "epochs": "250",
    "mini_batch_size": "32",
    "learning_rate": "0.00001",
    "dropout_rate": "0.05",
    "early_stopping_patience": "10" # stop if loss hasn't improved in 10 epochs
}

estimator.set_hyperparameters(**hyperparameters)

In [116]:
print(train_path)
print(test_path)
print(output_path)

data_channels = {"train": train_path, "test": test_path}

s3://bpdata_d4d38b6c-f257-4d83-9881-fe88b8f98a7c/sagemaker/train/deepar_training.json
s3://bpdata_d4d38b6c-f257-4d83-9881-fe88b8f98a7c/sagemaker/test/deepar_test.json
s3://bpdata_d4d38b6c-f257-4d83-9881-fe88b8f98a7c/sagemaker/output


In [117]:
estimator.fit(inputs=data_channels)

INFO:sagemaker:Creating training-job with name: bp-job-2018-02-04-00-14-40-208


.....................................................................
Arguments: train
[02/04/2018 00:20:31 INFO 140150847625024] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/default-input.json: {u'dropout_rate': u'0.05', u'cardinality': u'', u'test_quantiles': u'[0.5, 0.9]', u'_num_gpus': u'auto', u'learning_rate': u'0.001', u'_kvstore': u'auto', u'num_layers': u'3', u'embedding_dimension': u'', u'num_cells': u'40', u'_num_kv_servers': u'auto', u'mini_batch_size': u'32', u'likelihood': u'gaussian', u'early_stopping_patience': u''}
[02/04/2018 00:20:31 INFO 140150847625024] Reading provided configuration from /opt/ml/input/config/hyperparameters.json: {u'dropout_rate': u'0.05', u'learning_rate': u'0.00001', u'num_cells': u'40', u'prediction_length': u'24', u'epochs': u'250', u'time_freq': u'H', u'context_length': u'24', u'num_layers': u'2', u'mini_batch_size': u'32', u'likelihood': u'gaussian', u'early_stopping_patience': u'10'}
[02/04/2018 00:20

[02/04/2018 00:20:39 INFO 140150847625024] Epoch[4] Batch[5] avg_epoch_loss=7.090045
[02/04/2018 00:20:39 INFO 140150847625024] Epoch[4] Batch [5]#011Speed: 273.23 samples/sec#011loss=7.090045
[02/04/2018 00:20:39 INFO 140150847625024] Epoch[4] Batch[10] avg_epoch_loss=7.064467
[02/04/2018 00:20:39 INFO 140150847625024] Epoch[4] Batch [10]#011Speed: 374.84 samples/sec#011loss=7.033775
[02/04/2018 00:20:39 INFO 140150847625024] processed a total of 352 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1054.0051460266113, "sum": 1054.0051460266113, "min": 1054.0051460266113}}, "EndTime": 1517703639.730462, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1517703638.660435}

[02/04/2018 00:20:39 INFO 140150847625024] best epoch loss so far
[02/04/2018 00:20:39 INFO 140150847625024] Saved checkpoint to "/opt/ml/model/state_5a09f586-fb81-469f-b30e-87aba7d11361-0000.params"
#metrics {"Metrics": {"state.serialize.time": {"count

[02/04/2018 00:20:48 INFO 140150847625024] Epoch[13] Batch[5] avg_epoch_loss=6.501285
[02/04/2018 00:20:48 INFO 140150847625024] Epoch[13] Batch [5]#011Speed: 352.89 samples/sec#011loss=6.501285
[02/04/2018 00:20:49 INFO 140150847625024] processed a total of 320 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 996.9680309295654, "sum": 996.9680309295654, "min": 996.9680309295654}}, "EndTime": 1517703649.388432, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1517703648.378538}

[02/04/2018 00:20:49 INFO 140150847625024] best epoch loss so far
[02/04/2018 00:20:49 INFO 140150847625024] Saved checkpoint to "/opt/ml/model/state_8c6612fb-fc74-4385-8ebf-4a1321a7b1d4-0000.params"
#metrics {"Metrics": {"state.serialize.time": {"count": 1, "max": 55.47690391540527, "sum": 55.47690391540527, "min": 55.47690391540527}}, "EndTime": 1517703649.444176, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/Deep

[02/04/2018 00:20:59 INFO 140150847625024] Epoch[22] Batch[10] avg_epoch_loss=6.207907
[02/04/2018 00:20:59 INFO 140150847625024] Epoch[22] Batch [10]#011Speed: 343.60 samples/sec#011loss=6.172213
[02/04/2018 00:20:59 INFO 140150847625024] processed a total of 352 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1053.8179874420166, "sum": 1053.8179874420166, "min": 1053.8179874420166}}, "EndTime": 1517703659.125933, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1517703658.060121}

[02/04/2018 00:20:59 INFO 140150847625024] best epoch loss so far
[02/04/2018 00:20:59 INFO 140150847625024] Saved checkpoint to "/opt/ml/model/state_7ba08f57-b25e-4ad6-a5f8-81cdd2d544e3-0000.params"
#metrics {"Metrics": {"state.serialize.time": {"count": 1, "max": 62.53790855407715, "sum": 62.53790855407715, "min": 62.53790855407715}}, "EndTime": 1517703659.188657, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS

...

[02/04/2018 00:24:29 INFO 140150847625024] processed a total of 320 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1038.6428833007812, "sum": 1038.6428833007812, "min": 1038.6428833007812}}, "EndTime": 1517703869.335904, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1517703868.280446}

[02/04/2018 00:24:29 INFO 140150847625024] best epoch loss so far
[02/04/2018 00:24:29 INFO 140150847625024] Saved checkpoint to "/opt/ml/model/state_af2339c7-53e2-444d-9362-c33f8c161e4f-0000.params"
#metrics {"Metrics": {"state.serialize.time": {"count": 1, "max": 14.554977416992188, "sum": 14.554977416992188, "min": 14.554977416992188}}, "EndTime": 1517703869.350688, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1517703869.335985}

[02/04/2018 00:24:29 INFO 140150847625024] Epoch[214] Batch[0] avg_epoch_loss=4.593349
[02/04/2018 00:24:29 INFO 140150847625024] Epoch[214] Batch[5] 

[02/04/2018 00:24:39 INFO 140150847625024] processed a total of 320 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1068.4750080108643, "sum": 1068.4750080108643, "min": 1068.4750080108643}}, "EndTime": 1517703879.341885, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1517703878.261226}

[02/04/2018 00:24:39 INFO 140150847625024] loss did not improve for 3 epochs
[02/04/2018 00:24:39 INFO 140150847625024] Epoch[223] Batch[0] avg_epoch_loss=44.241039
[02/04/2018 00:24:39 INFO 140150847625024] Epoch[223] Batch[5] avg_epoch_loss=11.154683
[02/04/2018 00:24:39 INFO 140150847625024] Epoch[223] Batch [5]#011Speed: 291.83 samples/sec#011loss=11.154683
[02/04/2018 00:24:40 INFO 140150847625024] processed a total of 320 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 997.251033782959, "sum": 997.251033782959, "min": 997.251033782959}}, "EndTime": 1517703880.353528, "Dimensions": {"Host": "algo-1", "Operation

[02/04/2018 00:24:49 INFO 140150847625024] Epoch[232] Batch[5] avg_epoch_loss=4.495730
[02/04/2018 00:24:49 INFO 140150847625024] Epoch[232] Batch [5]#011Speed: 308.34 samples/sec#011loss=4.495730
[02/04/2018 00:24:49 INFO 140150847625024] processed a total of 320 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1018.5708999633789, "sum": 1018.5708999633789, "min": 1018.5708999633789}}, "EndTime": 1517703889.831333, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1517703888.799898}

[02/04/2018 00:24:49 INFO 140150847625024] best epoch loss so far
[02/04/2018 00:24:49 INFO 140150847625024] Saved checkpoint to "/opt/ml/model/state_b80ca4b8-08cb-4aec-b274-1cb24ff1a49f-0000.params"
#metrics {"Metrics": {"state.serialize.time": {"count": 1, "max": 51.9871711730957, "sum": 51.9871711730957, "min": 51.9871711730957}}, "EndTime": 1517703889.883492, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/De

[02/04/2018 00:24:59 INFO 140150847625024] Epoch[241] Batch[5] avg_epoch_loss=4.378894
[02/04/2018 00:24:59 INFO 140150847625024] Epoch[241] Batch [5]#011Speed: 361.16 samples/sec#011loss=4.378894
[02/04/2018 00:24:59 INFO 140150847625024] processed a total of 320 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 962.0480537414551, "sum": 962.0480537414551, "min": 962.0480537414551}}, "EndTime": 1517703899.580714, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1517703898.607053}

[02/04/2018 00:24:59 INFO 140150847625024] best epoch loss so far
[02/04/2018 00:24:59 INFO 140150847625024] Saved checkpoint to "/opt/ml/model/state_68420717-6702-4f79-921c-079873ae4e1b-0000.params"
#metrics {"Metrics": {"state.serialize.time": {"count": 1, "max": 65.66691398620605, "sum": 65.66691398620605, "min": 65.66691398620605}}, "EndTime": 1517703899.646578, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/De

#metrics {"Metrics": {"get_graph.time": {"count": 1, "max": 791.3470268249512, "sum": 791.3470268249512, "min": 791.3470268249512}}, "EndTime": 1517703909.53746, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1517703908.745986}

[02/04/2018 00:25:10 INFO 140150847625024] Number of GPUs being used: 0
#metrics {"Metrics": {"finalize.time": {"count": 1, "max": 1319.969892501831, "sum": 1319.969892501831, "min": 1319.969892501831}}, "EndTime": 1517703910.06606, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1517703909.537509}

[02/04/2018 00:25:10 INFO 140150847625024] Serializing to /opt/ml/model/model_algo-1
[02/04/2018 00:25:10 INFO 140150847625024] Saved checkpoint to "/opt/ml/model/model_algo-1-0000.params"
#metrics {"Metrics": {"model.serialize.time": {"count": 1, "max": 93.0781364440918, "sum": 93.0781364440918, "min": 93.0781364440918}}, "EndTime": 1517703910.159222, "Dimensions"

In [122]:
job_name = estimator.latest_training_job.name
job_name

'bp-job-2018-02-04-00-14-40-208'

In [123]:
endpoint_name = sagemaker_session.endpoint_from_job(
    job_name=job_name,
    initial_instance_count=1,
    instance_type='ml.m4.xlarge',
    deployment_image=image_name,
    role=role
)

INFO:sagemaker:Creating model with name: bp-job-2018-02-04-00-14-40-208
INFO:sagemaker:Creating endpoint-config with name bp-job-2018-02-04-00-14-40-208
INFO:sagemaker:Creating endpoint with name bp-job-2018-02-04-00-14-40-208


------------------------------------------------------------------------------------------------------------------------------------!

In [125]:
predictor = sagemaker.predictor.RealTimePredictor(
    endpoint_name, 
    sagemaker_session=sagemaker_session, 
    content_type="application/json")

In [144]:
q1 = '0.1'         # compute p10 quantile
q2 = '0.9'         # compute p90 quantile
num_samples = 1  # predict 6 sample series
    
s = {"start": "2017-02-01 08:00:00", "target": bp_sistolic}
series = []
series.append(s)

configuration = {
        "output_types": ["samples"],
        "num_samples": num_samples}
http_data = {
        "instances": series, 
        "configuration": configuration
}
http_data = json.dumps(http_data)
print(http_data)

{"instances": [{"start": "2017-02-01 08:00:00", "target": [134, 138, 138, 124, 140, 134, 135, 141, 122, 137, 136, 130, 145, 121, 118, 118, 134, 116, 147, 112, 119, 146, 138, 142, 149, 115, 119, 133, 140, 149, 140, 136, 111, 117, 140, 145, 124, 132, 132, 141, 149, 128, 134, 132, 129, 119, 148, 119, 126, 121, 116, 119, 128, 127, 149, 123, 135, 147, 134, 122, 135, 127, 113, 130, 136, 145, 130, 146, 125, 123, 128, 125, 138, 119, 137, 143, 138, 127, 115, 123, 117, 135, 134, 132, 111, 111, 150, 129, 120, 117, 110, 113, 138, 141, 132, 132, 149, 147, 115, 146, 138, 129, 133, 129, 139, 111, 147, 124, 124, 122, 138, 141, 135, 125, 130, 138, 114, 128, 131, 136, 140, 113, 149, 126, 148, 149, 146, 146, 117, 143, 129, 142, 145, 117, 115, 144, 113, 146, 146, 118, 124, 116, 129, 124, 146, 133, 131, 132, 148, 118, 144, 122, 136, 111, 123, 119, 125, 123, 119, 149, 127, 128, 114, 128, 146, 144, 135, 113, 136, 146, 117, 135, 150, 119, 116, 143, 149, 147, 140, 117, 114, 120, 115, 118, 114, 135, 114, 140, 1

In [145]:
result = predictor.predict(http_data)
result.decode('utf-8')

'{"predictions":[{"samples":[[125.4389038086,115.7813720703,128.1891784668,125.4847869873,102.7653808594,98.6500930786,133.0986328125,95.2591247559,136.8067779541,112.9621810913,117.986000061,99.4025115967,114.7673110962,131.3999023438,140.259475708,167.5045013428,191.4559326172,168.1932525635,125.5912628174,83.216293335,122.5363845825,104.8993148804,96.9972686768,119.0280456543]]}]}'